# Call rolling horizon functions

In [1]:
from scen_generation_new_generate import generateForecast
import numpy as np
import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.colors import LightSource
#%matplotlib notebook

In [2]:
from upside import runDP, plotVF, buildOfferCurves, runThreeStage, runThreeStageBorder, plotOfferCurves
#from upside import offer_curve_as_fcn

In [3]:
#%qtconsole

Load data:

In [8]:
profitDAYS = []
penalDAYS = []


for day in range(28,60): #60

    stTime = time.time()
    # Set day
    print("Starting new loop ...................")
    
    which_day = day 
    num_scenarios = 4
    Bm_dict1, Prob_ay1 = generateForecast(which_day, num_scenarios, plot=False)
    Bm_dict2, _ = generateForecast(which_day+1, num_scenarios, plot=False)
    
    # Replace negative bm prices with a zero
    for hora in range(1,25):
        for k,v in Bm_dict1['hour{}'.format(hora)]['lambda_bm_scend'].items():
            if v<0:
                Bm_dict1['hour{}'.format(hora)]['lambda_bm_scend'][k] = 1
                
    
    print("")
    print("Loaded data for day {}.".format(which_day))
    
    # Set parameters:
    PARAMS = {
        'RES'   : 10,        # resolution of discretization (10)
        'MDP'   : -10**5,    # penalty in DP model
        'MST'   : 10**5,     # penalty in stochastic model
        'QDA'   : 30,        # energy contracted in the DA (30)
        'C0'    :  0,        # thermal unit fixed op. cost
        'MC'    : 25,        # diesel marginal cost (25)
        'CST'   :  5,        # diesel start-up cost
        'CSH'   :  5,        # diesel shut-down cost
        'LMIN'  :  0,        # storage min. level
        'LMAX'  : 80,        # storage max. level (80)
        'DMIN'  :  0,        # diesel min. output
        'DMAX'  : 70,        # diesel max. output (70)
        'DRAMP' : 30,        # diesel max ramp up/dw (70)
        'PUP'   : 30,        # max. battery charge (30)
        'PDW'   : 30,        # max. battery discharge (30)
        'L0'    : 30,        # storage level in start of t=0 (3-stage model) (30)
        'D0'    : 30,        # diesel status in start of t=0 (3-stage model) (30)
        'U0'    :  1,        # uc status of diesel in start of t=0 (3-stage model)
        'ETA'   :  1,        # storage charging efficiency
        'VFfac' : 0.5,       # Value function factor in objective
    }
    
    # Run the day
    useVF=True
    log=True

    offcurves = buildOfferCurves()
    L0 = PARAMS['L0']
    D0 = PARAMS['D0']
    profitAll = []
    penalAll = np.array([0,0,0,0])
    
    for h in range(1,25):
        if h <= 21:

            # DP
            CC, FF = runDP(hour=h, params=PARAMS, data=Bm_dict1, data2=Bm_dict2, probMat=Prob_ay1)
            #CC = [0,0,0,0,0,0]
            
            # Three-stage 
            oc_new, newL0, newD0, profit_new, profit0, penal = runThreeStage(hour=h, params=PARAMS, data=Bm_dict1, probMat=Prob_ay1, 
                                                               offerCurves=offcurves, L0=L0,D0=D0, CC=CC, useVF=useVF, log=log)

            # Update offer curves
            offcurvesAux = {}
            offcurvesAux[0] = offcurves[1]
            offcurvesAux[1] = offcurves[2]
            offcurvesAux[2] = oc_new
            offcurves = offcurvesAux

            # Update 'L0' and 'D0'
            L0 = newL0
            D0 = newD0

            # Update penalties
            penalAll += np.array(penal)

            # Save profit
            profitAll.append(profit0)
            
            # plot
            #plotOfferCurves(offerCurves=oc_new, hour=h, data=Bm_dict1)


        elif h >21:
            # Three-stage 
            oc_new, newL0, newD0, profit_new, profit0, penal = runThreeStageBorder(hour=h, params=PARAMS, data=Bm_dict1, probMat=Prob_ay1, 
                                                               offerCurves=offcurves, L0=L0,D0=D0, CC=CC, useVF=False, log=log)

            # Update offer curves
            offcurvesAux = {}
            offcurvesAux[0] = offcurves[1]
            offcurvesAux[1] = offcurves[2]
            offcurvesAux[2] = oc_new
            offcurves = offcurvesAux

            # Update 'L0' and 'D0'
            L0 = newL0
            D0 = newD0

            # Update penalties
            penalAll += np.array(penal)

            # Save profit
            profitAll.append(profit0)
    
    
    profitDAYS.append(profitAll)
    penalDAYS.append(penalAll)
    
    print("RH for day {} was succesful.".format(which_day))
    print("")
    endTime = time.time() - stTime
    print("Running time for day: {0:.3f} seconds.".format(endTime))


    
    

Starting new loop ...................

Loaded data for day 28.
 
Running time (DP): 3.311 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.14 secs.
Problem is Ok and feasible.
Solution for hour 1
Profit: -13916.491177055146, sumVF: 24366.07332795312
Profit hour 1 = 680.6787294454301 
BM price hour 1: 67.50098306880378
DA price hour 1: 51.032213188866365
qda=30, qup=0.0, qdw=0.0
D0=30, L0=30
d=33.61150664842722, l=33.61150664842723, pup=3.6115066484272305, pdw=0.0
 
Running time (DP): 3.089 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.119 secs.
Problem is Ok and feasible.
Solution for hour 2
Profit: -10136.583767111739, sumVF: 22254.476781484354
Profit hour 2 = 332.42446610126626 
BM price hour 2: 70.37990402542272
DA price hour 2: 54.45398298195403
qda=30, qup=0.0, qdw=0.0
D0=33.61150664842722, L0=33.61150664842723
d=51.64780093377499, l=55.25930758220222, pup=21.647800933774988, p

 
Running time (DP): 3.075 seconds.
Running time (LS): 0.006 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.13 secs.
Problem is Ok and feasible.
Solution for hour 18
Profit: -6684.692770298364, sumVF: 14342.951184115132
Profit hour 18 = 757.879531536958 
BM price hour 18: 75.65564642339183
DA price hour 18: 75.59598438453668
qda=30, qup=0.0, qdw=0.0
D0=70.0, L0=2.8673355697252143e-15
d=59.99999999994553, l=29.999999999945523, pup=30.0, pdw=5.448086426440568e-11
 
Running time (DP): 3.077 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.111 secs.
Problem is Ok and feasible.
Solution for hour 19
Profit: -403.9192001049978, sumVF: 12125.447048610631
Profit hour 19 = 3985.8807382211535 
BM price hour 19: 84.79658151851073
DA price hour 19: 80.32600193704678
qda=30, qup=39.99999999987843, qdw=0.0
D0=59.99999999994553, L0=29.999999999945523
d=70.0, l=30.00000000006708, pup=30.0, pdw=29.999999999878444
 
Running time (DP): 

 
Running time (DP): 2.950 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.136 secs.
Problem is Ok and feasible.
Solution for hour 12
Profit: -11094.520065279, sumVF: 16667.282839189484
Profit hour 12 = 323.37961724538604 
BM price hour 12: 46.33469270190568
DA price hour 12: 49.01614602185333
qda=30, qup=0.0, qdw=3.014624824781713e-10
D0=31.563863730512907, L0=5.634722640218418
d=45.484190535631114, l=21.11891317615103, pup=15.484190535932612, pdw=0.0
 
Running time (DP): 2.942 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.117 secs.
Problem is Ok and feasible.
Solution for hour 13
Profit: -10781.06859169569, sumVF: 17271.152161383616
Profit hour 13 = 828.03567595526 
BM price hour 13: 42.49127757818148
DA price hour 13: 50.85862615748823
qda=30, qup=0.0, qdw=1.5375266423242593e-09
D0=45.484190535631114, L0=21.11891317615103
d=27.508924345796647, l=18.627837523485255, pup=27.508924

 
Solver status: ok
Problem condition:  optimal
Time:  0.147 secs.
Problem is Ok and feasible.
Solution for hour 5
Profit: -4677.144836767609, sumVF: 14448.181850615652
Profit hour 5 = 6840.489246043177 
BM price hour 5: 54.51573763146625
DA price hour 5: 47.12131443315917
qda=30, qup=69.99999999936784, qdw=0.0
D0=70.0, L0=39.99999999981674
d=70.0, l=9.999999999816755, pup=1.7763568394002505e-14, pdw=30.0
 
Running time (DP): 3.210 seconds.
Running time (LS): 0.007 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.161 secs.
Problem is Ok and feasible.
Solution for hour 6
Profit: -6765.973111330525, sumVF: 14099.580517722328
Profit hour 6 = 4534.50756492614 
BM price hour 6: 51.98763826085632
DA price hour 6: 44.56636087329434
qda=30, qup=49.99999999559155, qdw=0.0
D0=70.0, L0=9.999999999816755
d=70.0, l=1.3284825430055137e-15, pup=0.0, pdw=9.999999999816753
 
Running time (DP): 3.058 seconds.
Running time (LS): 0.006 seconds.
 
Solver status: ok
Problem condition:  opti

 
Solver status: ok
Problem condition:  optimal
Time:  0.049 secs.
Problem is Ok and feasible.
Solution for hour 22
Profit: -984.5789403242386
Profit hour 22 = 3063.1121335064045 
BM price hour 22: 32.742735628226136
DA price hour 22: 30.880538816420085
qda=30, qup=39.999999991658335, qdw=0.0
D0=70.0, L0=1.3885469351902904e-16
d=70.0, l=0.0, pup=0.0, pdw=0.0
 
Solver status: ok
Problem condition:  optimal
Time:  0.047 secs.
Problem is Ok and feasible.
Solution for hour 23
Profit: -503.32813260509556
Profit hour 23 = 3586.792239734881 
BM price hour 23: 36.4074412940532
DA price hour 23: 33.81628550697826
qda=30, qup=39.99999999965545, qdw=0.0
D0=70.0, L0=0.0
d=70.0, l=0.0, pup=0.0, pdw=0.0
 
Solver status: ok
Problem condition:  optimal
Time:  0.043 secs.
Problem is Ok and feasible.
Solution for hour 24
Profit: -186.57769170568508
Profit hour 24 = 4004.7753353483195 
BM price hour 24: 39.33555740890525
DA price hour 24: 36.115288408137985
qda=30, qup=39.99999999988058, qdw=0.0
D0=70.0,

...

 
Solver status: ok
Problem condition:  optimal
Time:  0.099 secs.
Problem is Ok and feasible.
Solution for hour 14
Profit: -8841.200235120881, sumVF: 12307.04568753351
Profit hour 14 = 1287.0798947592425 
BM price hour 14: 57.0025477767973
DA price hour 14: 61.7260785879655
qda=30, qup=0.0, qdw=1.7146128238274055e-11
D0=32.44661747356985, L0=14.092295747739135
d=22.188098514641005, l=6.280394262397286, pup=22.18809851465815, pdw=30.0
 
Running time (DP): 3.244 seconds.
Running time (LS): 0.006 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.103 secs.
Problem is Ok and feasible.
Solution for hour 15
Profit: -9689.566758668117, sumVF: 13337.164430229448
Profit hour 15 = 1145.1689770032544 
BM price hour 15: 53.175439649337264
DA price hour 15: 61.55806477823853
qda=30, qup=0.0, qdw=2.096905669786742e-12
D0=22.188098514641005, L0=6.280394262397286
d=27.662918653380338, l=3.9433129157797215, pup=27.662918653382434, pdw=30.0
 
Running time (DP): 3.234 seconds.
Running tim

 
Running time (DP): 3.531 seconds.
Running time (LS): 0.006 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.124 secs.
Problem is Ok and feasible.
Solution for hour 8
Profit: -11686.135637092433, sumVF: 21949.063490188782
Profit hour 8 = 1000.9977889794025 
BM price hour 8: 39.95295095183227
DA price hour 8: 39.18757028255703
qda=30, qup=39.99999999973428, qdw=0.0
D0=70.0, L0=8.679785174462788e-16
d=69.99999999999997, l=4.493477274294139e-14, pup=4.406679422549511e-14, pdw=0.0
 
Running time (DP): 3.306 seconds.
Running time (LS): 0.006 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.127 secs.
Problem is Ok and feasible.
Solution for hour 9
Profit: -14207.173900599571, sumVF: 22427.250212551353
Profit hour 9 = 186.95692592039427 
BM price hour 9: 46.096355863508215
DA price hour 9: 48.23189752957171
qda=30, qup=0.0, qdw=0.0
D0=69.99999999999997, L0=4.493477274294139e-14
d=49.99999999865803, l=19.999999998652008, pup=19.999999998651962, pdw=0.0
 
Runn


Loaded data for day 58.
 
Running time (DP): 3.271 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.172 secs.
Problem is Ok and feasible.
Solution for hour 1
Profit: -7748.867492683608, sumVF: 12249.491107513604
Profit hour 1 = 1040.0873519573515 
BM price hour 1: 44.54432055661542
DA price hour 1: 35.002915447402074
qda=30, qup=0.0, qdw=0.0
D0=30, L0=30
d=4.458585972346753e-06, l=4.458585972346753e-06, pup=4.458585973310392e-06, pdw=30.0
 
Running time (DP): 3.365 seconds.
Running time (LS): 0.006 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.134 secs.
Problem is Ok and feasible.
Solution for hour 2
Profit: -8840.612581020825, sumVF: 13865.869685999378
Profit hour 2 = 176.02864356093949 
BM price hour 2: 37.819344899842804
DA price hour 2: 31.200958498523732
qda=30, qup=0.0, qdw=0.0
D0=4.458585972346753e-06, L0=4.458585972346753e-06
d=30.000004455398273, l=8.913984244546168e-06, pup=30.0, pdw=29.999995544601727
 


 
Running time (DP): 3.140 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.129 secs.
Problem is Ok and feasible.
Solution for hour 17
Profit: -8540.301958819344, sumVF: 12953.964961779444
Profit hour 17 = 379.3476704656273 
BM price hour 17: 61.015550170700635
DA price hour 17: 61.50265792074083
qda=30, qup=0.0, qdw=1.5506102954284325e-11
D0=40.00008359882174, L0=10.00008359882174
d=58.22928268620754, l=38.22936628504479, pup=28.22928268622305, pdw=0.0
 
Running time (DP): 3.138 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.129 secs.
Problem is Ok and feasible.
Solution for hour 18
Profit: -3884954.8932956886, sumVF: 12226.081518647785
Profit hour 18 = 3909.040305745177 
vi_plus[1,1]=21.770633692919922
vi_plus[1,2]=21.77063479389951
vi_plus[1,3]=21.770634793963595
vi_plus[2,1]=29.999999998290026
vi_plus[2,2]=29.99999999829007
vi_plus[2,3]=29.999999998227047
vi_plus[2,4]=29.99999999

 
Running time (DP): 3.201 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.111 secs.
Problem is Ok and feasible.
Solution for hour 9
Profit: -8132.432672149503, sumVF: 10446.221391521349
Profit hour 9 = 975.4551942192217 
BM price hour 9: 56.767037894729306
DA price hour 9: 56.7670378947293
qda=30, qup=0.0, qdw=0.0
D0=24.413915645274223, L0=5.226512149989335
d=28.702237704703858, l=3.9287498546931907, pup=28.702237704703855, pdw=30.0
 
Running time (DP): 3.241 seconds.
Running time (LS): 0.005 seconds.
 
Solver status: ok
Problem condition:  optimal
Time:  0.118 secs.
Problem is Ok and feasible.
Solution for hour 10
Profit: -8118.233957236403, sumVF: 10036.976043991333
Profit hour 10 = 1179.1231676307657 
BM price hour 10: 64.76844839988729
DA price hour 10: 64.76844839988729
qda=30, qup=0.0, qdw=0.0
D0=28.702237704703858, L0=3.9287498546931907
d=30.157211374164834, l=4.085961228858029, pup=0.157211374164838, pdw=0.0
 
Running time (D

In [9]:
%store profitDAYS > profitDAYS.txt
%store penalDAYS > penalDAYS.txt

Writing 'profitDAYS' (list) to file 'profitDAYS.txt'.
Writing 'penalDAYS' (list) to file 'penalDAYS.txt'.


In [ ]:
len(profitDAYS)